Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fnormal-crit-vs-another-bite-207994%2Fnormal_crit_vs_another_bite.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/normal-crit-vs-another-bite-207994)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“Over time, how does doubling dice on a crit compare vs having another action?”](https://rpg.stackexchange.com/a/208013/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H, P
from dyce.evaluation import HResult, expandable
from enum import Enum, IntEnum
from fractions import Fraction

class HitOutcome(IntEnum):
    CRIT_MISS = -1
    MISS = 0
    HIT = 1
    CRIT_HIT = 2

class Advantage(Enum):
    DISADVANTAGE = "Disadv"
    NORMAL = "Normal"
    ADVANTAGE = "Adv"

d20 = H(20)
MIN_DMG = 0  # See <https://www.dndbeyond.com/sources/basic-rules/combat#DamageRolls>

def hit_outcomes_from_target_bonus(
    target: int,
    hit_bonus: int,
    advantage: Advantage = Advantage.NORMAL,
    crit_hit_threshold: int = max(d20),
    crit_miss_threshold: int = min(d20),
    enforce_crit_misses: bool = True,
) -> H:
    def _outcome_count_gen():
        for outcome, count in d20.items():
            if outcome >= crit_hit_threshold:
                yield HitOutcome.CRIT_HIT, count
            elif outcome == crit_miss_threshold and enforce_crit_misses:
                yield HitOutcome.CRIT_MISS, count
            elif outcome + hit_bonus >= target:
                yield HitOutcome.HIT, count
            else:
                yield HitOutcome.MISS, count

    outcomes = H(_outcome_count_gen())
    if advantage is Advantage.NORMAL:
        return outcomes
    elif advantage is Advantage.ADVANTAGE:
        return H({
            HitOutcome(outcome): count
            for outcome, count in (2 @ P(outcomes)).h(-1).items()
        })
    elif advantage is Advantage.DISADVANTAGE:
        return H({
            HitOutcome(outcome): count
            for outcome, count in (2 @ P(outcomes)).h(0).items()
        })
    else:
        assert False, "shouldn't ever be here"

def normal_crit(dmg_die: H, dmg_bonus: int) -> H:
    return H(
        (max(outcome, MIN_DMG), count)
        for outcome, count in (2 @ dmg_die + dmg_bonus).items()
    )

def double_dmg_house_rule(dmg_die: H, dmg_bonus: int) -> H:
    return H(
        (max(outcome, MIN_DMG), count)
        for outcome, count in ((dmg_die + dmg_bonus) * 2).items()
    )

def another_bite(
    dmg_die: H,
    target: int,
    hit_bonus: int,
    dmg_bonus: int,
    advantage: Advantage = Advantage.NORMAL,
    crit_hit_threshold: int = max(d20),
    crit_miss_threshold: int = min(d20),
    enforce_crit_misses_on_rerolls: bool = True,
    dmg_bonus_on_rerolls: bool = False,
) -> H:
    @expandable
    def _explode_crit_hits(result: HResult):
        # This allows us to add our damage bonus to each roll if dmg_bonus_on_rerolls is
        # True
        reroll_dmg_bonus = dmg_bonus if dmg_bonus_on_rerolls else 0
        if result.outcome is HitOutcome.CRIT_HIT:
            # We hit *another* crit, so include another damage die, possibly another
            # damage bonus (see above), and whatever *other* crits we may encounter as we
            # keep rolling
            return dmg_die + reroll_dmg_bonus + _explode_crit_hits(result.h)  # result.h is hit_outcomes
        elif result.outcome is HitOutcome.HIT:
            # We got a hit after our first crit, so include another damage die and
            # possibly another damage bonus (see above)
            return dmg_die + reroll_dmg_bonus
        else:  # treats HitOutcome.MISS and HitOutcome.CRIT_MISS as equivalent
            # We missed on our re-roll, so don't include any additional damage
            return H({0: 1})  # no (more) damage

    hit_outcomes = hit_outcomes_from_target_bonus(
        target,
        hit_bonus,
        advantage,
        crit_hit_threshold,
        crit_miss_threshold,
        enforce_crit_misses_on_rerolls,
    )
    # Start us off with our damage die and damage bonus and whatever else we can re-roll,
    # but make sure that damage has a floor of zero (to handle negative damage bonuses)
    return H(
        (max(outcome, MIN_DMG), count)
        for outcome, count in
        (dmg_die + _explode_crit_hits(hit_outcomes, limit=Fraction(1, 10_000)) + dmg_bonus).items()
    )

In [3]:
DMG_DICE = {
    "d4": H(4),
    "d6": H(6),
    "d8": H(8),
    "d10": H(10),
    "d12": H(12),
    "d20": H(20),
    "2d4": 2 @ H(4),
    "2d6": 2 @ H(6),
    "2d8": 2 @ H(8),
    # ...
}
SELECTED_DMG_DICE = list(DMG_DICE)[:5]  # start by selecting first five values (whatever they are)

In [4]:
# Interactive UI code
from anydyce import HPlotterChooser
from anydyce.viz import PlotWidgets
from IPython.display import display
from ipywidgets import widgets

NORMAL_CRIT_MECH = "normal crit"
DOUBLE_DMG_MECH = "double all dmg"
ANOTHER_BITE_MECH = "another bite @AC"

def _display(
    selected_dmg_dice: list[str],
    target_range: tuple[int, int],
    advantage_value: str,
    hit_bonus: int,
    dmg_bonus: int,
    crit_range: tuple[int, int],
    crit_misses_on_rerolls: bool,
    dmg_bonus_on_rerolls: bool,
) -> None:
    results = []
    target_lo, target_hi = target_range
    target_hi += 1  # exclusive upper bound
    advantage = Advantage(advantage_value)
    crit_miss_threshold, crit_hit_threshold = crit_range
    num_charts = 2 + (target_hi - target_lo)  # normal crit, double dmg, and one for each re-roll target
    fillers = -num_charts % chooser._plot_widgets.burst_columns.value
    for dmg_die_name in selected_dmg_dice:
        dmg_die = DMG_DICE[dmg_die_name]
        normal_crit_res = normal_crit(dmg_die, dmg_bonus)
        double_dmg_res = double_dmg_house_rule(dmg_die, dmg_bonus)
        results.append(
            (
                f"{dmg_die_name} {NORMAL_CRIT_MECH}\n({normal_crit_res.mean():0.3f})",
                normal_crit_res,
            )
        )
        results.append(
            (
                f"{dmg_die_name} {DOUBLE_DMG_MECH}\n({double_dmg_res.mean():0.3f})",
                double_dmg_res,
                normal_crit_res,
            )
        )
        for target in range(target_lo, target_hi):
            another_bite_res = another_bite(
                dmg_die,
                target,
                hit_bonus,
                dmg_bonus,
                advantage,
                crit_hit_threshold,
                crit_miss_threshold,
                crit_misses_on_rerolls,
                dmg_bonus_on_rerolls,
            )
            mech_hdr = f"{ANOTHER_BITE_MECH}{target}"
            results.append(
                (
                    f"{dmg_die_name} {mech_hdr}\n({another_bite_res.mean():0.3f})",
                    another_bite_res,
                    normal_crit_res,
                )
            )
        results.extend((f"", H({})) for _ in range(fillers))
    chooser.update_hs(results)

dmg_dice_widget = widgets.SelectMultiple(
    options=list(DMG_DICE),
    value=SELECTED_DMG_DICE,
    description="Dmg Dice",
)
target_range_widget = widgets.IntRangeSlider(
    value=(13, 19),
    min=2,
    max=30,
    step=1,
    continuous_update=False,
    description="AC Range",
)
hit_bonus_widget = widgets.BoundedIntText(
    value=11,
    min=-10,
    max=20,
    step=1,
    description="Hit Bonus",
)
dmg_bonus_widget = widgets.BoundedIntText(
    value=5,
    min=-10,
    max=20,
    step=1,
    description="Dmg Bonus",
)
advantage_value_widget = widgets.SelectionSlider(
    options=[e.value for e in Advantage],
    value=Advantage.NORMAL.value,
    continuous_update=False,
    description="Advantage",
)
crit_range_widget = widgets.IntRangeSlider(
    value=(min(d20), max(d20)),
    min=min(d20),
    max=max(d20),
    step=1,
    description="Crits At",
)
crit_misses_on_rerolls_widget = widgets.Checkbox(
    value=True,
    description="Crit Misses Each Re-roll",
)
dmg_bonus_on_rerolls_widget = widgets.Checkbox(
    value=True,
    description="Dmg Bonus Each Re-roll",
)
chooser = HPlotterChooser(
    plot_widgets=PlotWidgets(
        initial_burst_columns=5,
        initial_burst_zero_fill_normalize=True,
        initial_resolution=18,
    )
)

display(
    widgets.HBox([
        widgets.VBox([
            dmg_dice_widget,
            target_range_widget,
        ]),
        widgets.VBox([
            hit_bonus_widget,
            dmg_bonus_widget,
            dmg_bonus_on_rerolls_widget,
        ]),
        widgets.VBox([
            advantage_value_widget,
            crit_range_widget,
            crit_misses_on_rerolls_widget,
        ]),
    ]),
    widgets.interactive_output(
        _display,
        {
            "selected_dmg_dice": dmg_dice_widget,
            "target_range": target_range_widget,
            "hit_bonus": hit_bonus_widget,
            "dmg_bonus": dmg_bonus_widget,
            "dmg_bonus_on_rerolls": dmg_bonus_on_rerolls_widget,
            "advantage_value": advantage_value_widget,
            "crit_range": crit_range_widget,
            "crit_misses_on_rerolls": crit_misses_on_rerolls_widget,
        },
    ),
)

chooser.interact()

Output()